In [2]:
import numpy as np
import statsmodels.api as sm

In [3]:
num = 1000 
 
event_time = int(num / 2) 
 
R_market = np.random.normal(0, 1, num) + np.arange(num) / num 
 
R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == int(num / 2) + 1) * 2 
 
results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit() 
 
alpha, beta = results.params 
 
resid = R_target - results.predict(sm.add_constant(R_market)) 
 
print(resid[event_time + 1] / resid[:event_time].std(ddof = 2)) 

1.8873817808190254


In [4]:
success = 0
n_sim = 1000
num = 1000 
 
event_time = int(num / 2) 

for _ in range(n_sim):
 
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num 
 
    R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == int(num / 2) + 1) * 2 
 
    results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit() 
 
    alpha, beta = results.params 
 
    resid = R_target - results.predict(sm.add_constant(R_market)) 
    tval = resid[event_time + 1]/ resid[:event_time].std(ddof=2)
    if abs(tval) > 1.96:
        success += 1
print("Estimated power", success / n_sim)

Estimated power 0.516


In [5]:
#q2
num = 1000
event_time = int(num/2)

np.random.seed(0)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num 
 
R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == event_time + 1) * 2 

success = 0
tested_times = 0

for fictitious_event_time in range(100, num-2):
    result = sm.OLS(
        R_target[:fictitious_event_time],
        sm.add_constant(R_market[:fictitious_event_time])
    ).fit()
    resid = R_target - results.predict(sm.add_constant(R_market))
    tval = resid[fictitious_event_time] / resid[:fictitious_event_time].std(ddof=2)
    if abs(tval) > 1.96:
        success += 1
    tested_times += 1
print('Fraction significant:', success/tested_times)

Fraction significant: 0.0467706013363029


In [6]:
# q3
num = 1000
event_time = int(num/2)
n_sim = 500
results = []

for seed in range(n_sim):
    np.random.seed(seed)
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num
    R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == event_time + 1) * 2
    
    model = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit()
    resid = R_target - model.predict(sm.add_constant(R_market))
    t_real = abs(resid[event_time + 1]/resid[:event_time].std(ddof=2))

# 20 before and 20 after
    count = 0
    total = 0
    for offset in range(-20, 21):
        fake_time = event_time + offset
        if fake_time == event_time + 1 or fake_time < 100 or fake_time > num - 3:
            continue
        model_fake = sm.OLS(R_target[:fake_time], sm.add_constant(R_market[:fake_time])).fit()
        resid_fake = R_target - model_fake.predict(sm.add_constant(R_market))
        t_fake = abs(resid_fake[fake_time]/resid_fake[:fake_time].std(ddof=2))
        if t_fake > t_real:
            count += 1
        total += 1
    results.append(count/total)
print("Average fraction of placebo t_value > real t_values:",np.mean(results))

Average fraction of placebo t_value > real t_values: 0.14809999999999998


In [7]:
def make_error(corr_const, num): 
 
 
  sigma = 5 * 1 / np.sqrt((1 - corr_const)**2 / (1 - corr_const**2)) 
 
 
  err = list() 
 
 
  prev = np.random.normal(0, sigma) 
 
 
  for n in range(num): 
 
 
    prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, sigma) 
 
 
    err.append(prev) 
 
 
  return np.array(err) 

In [ ]:
#q4
def make_error(corr_const, num):
    sigma = 5 * 1 / np.sqrt((1 - corr_const)**2 / (1 - corr_const**2)) 
    err = []
    prev = np.random.normal(0, sigma)
    for n in range(num): 
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, sigma) 
        err.append(prev) 
    return np.array(err)

num = 1000
event_time = int(num / 2)
np.random.seed(0)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num
error = make_error(0.9, num)
R_target = 2 + R_market + error + (np.arange(num) == event_time + 1) * 2

success = 0
tested_times = 0

for fictitious_event_time in range(100, num - 2):
    results = sm.OLS(
        R_target[:fictitious_event_time],
        sm.add_constant(R_market[:fictitious_event_time])
    ).fit()
    resid = R_target - results.predict(sm.add_constant(R_market))
    tval = resid[fictitious_event_time]/resid[:fictitious_event_time].std(ddof=2)

    if abs(tval) > 1.96:
        success += 1
    tested_times += 1
print("Fraction significant (placebo, autocorrelated errors):", success / tested_times)

Fraction significant (placebo, autocorrelated errors): 0.036748329621380846


In [ ]:
# ?
def make_error(corr_const, num): 
    sigma = 5 * 1 / np.sqrt((1 - corr_const)**2 / (1 - corr_const**2)) 
    err = []
    prev = np.random.normal(0, sigma)
    for n in range(num):
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, sigma)
        err.append(prev)
    return np.array(err)

num = 1000
event_time = int(num / 2)
n_sim = 500
results = []

for seed in range(n_sim):
    np.random.seed(seed)
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num
    error = make_error(0.9, num)
    R_target = 2 + R_market + error + (np.arange(num) == event_time + 1) * 2
    
    model = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit()
    resid = R_target - model.predict(sm.add_constant(R_market))
    t_real = abs(resid[event_time + 1]/resid[:event_time].std(ddof=2))

    count = 0
    total = 0
    for offset in range(-20, 21):
        fake_time = event_time + offset
        if fake_time == event_time + 1 or fake_time < 100 or fake_time > num - 3:
            continue
        model_fake = sm.OLS(R_target[:fake_time], sm.add_constant(R_market[:fake_time])).fit()
        resid_fake = R_target - model_fake.predict(sm.add_constant(R_market))
        t_fake = abs(resid_fake[fake_time]/resid_fake[:fake_time].std(ddof=2))
        if t_fake > t_real:
            count += 1
        total += 1
    results.append(count/total)

print("Average fraction of placebo t_value > real t_value (autocorrelated errors):", np.mean(results))


Average fraction of placebo t_value > real t_value (autocorrelated errors): 0.4651
